In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
import pandas as pd
import os
#import pickle
from utility import *
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import GridSearchCV

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

def mlp_model_fn(features, labels, mode, params):
  """Model function for MLP."""
  
  #TODO: need works on this
  config = params


  # Input Layer
  
  input_layer = tf.reshape( features["x"], [-1, 256*3 ] )
  #print ('feature x', features["x"])
  #print ('feature x shape', features["x"].shape)
  #print ('reshape:', input_layer)
  #print ('reshape shape:', input_layer.shape)
  #trans = tf.string_to_number(input_layer)
  #print ('trans reshape:', trans)
  #print ('reshape shape:', input_layer.shape)


  # Dense Layers
  hidden1 = tf.layers.dense(inputs=input_layer, units=config['n_hidden1'], activation=None)
  normal_1 = tf.layers.batch_normalization(hidden1, training=mode == tf.estimator.ModeKeys.TRAIN)
  relu1 = tf.nn.relu(normal_1)
  drop_h1 = tf.layers.dropout(
      inputs=relu1, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)


  hidden2 = tf.layers.dense(inputs=drop_h1, units=config['n_hidden2'], activation=None)
  normal_2 = tf.layers.batch_normalization(hidden2, training=mode == tf.estimator.ModeKeys.TRAIN)
  relu2 = tf.nn.relu(normal_2)
  drop_h2 = tf.layers.dropout(
      inputs=relu2, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)
  if config['n_hidden3'] != None:
    hidden3 = tf.layers.dense(inputs=drop_h2, units=config['n_hidden3'], activation=None)
    normal_3 = tf.layers.batch_normalization(hidden3, training=mode == tf.estimator.ModeKeys.TRAIN)
    relu3 = tf.nn.relu(normal_3)
    drop_h3 = tf.layers.dropout(
        inputs=relu3, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=drop_h3, units=config['nclasses'])
  else:
    logits = tf.layers.dense(inputs=drop_h2, units=config['nclasses'])
    
  predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }


  

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=config['nclasses'])
  print (onehot_labels)
  print (logits)
  loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)


  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.001)

    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




In [3]:
unused_argv = ['pilot_mlp_newAPI.py', 'data_3p_companded.csv', 'model/', 1024, 512, 256]

filename = unused_argv[1]


if os.path.isfile('ft_to_idx.npy') and os.path.isfile('idx_to_ft.npy') and os.path.isfile('nclasses.npy') and os.path.isfile('group_data.npy'):
    ft_to_idx = np.load('ft_to_idx.npy')
    ft_to_idx = ft_to_idx.item()
    idx_to_ft = np.load('idx_to_ft.npy')
    idx_to_ft = idx_to_ft.item()
    nclasses = np.load('nclasses.npy')
    #f = open('group_data.pkl','r')
    #group_data = pickle.load('group_data.pkl')
    #group_data = np.load('group_data.npy') 
    group_data = np.load('toy_data.npy') # for proof of algo purpose, real use case should use the above line
    group_data = group_data.item()

else:
    ft_to_idx, idx_to_ft, nclasses, group_data = prepare_file(filename)
    np.save("ft_to_idx", ft_to_idx)
    np.save("idx_to_ft", idx_to_ft)
    np.save("nclasses", nclasses)
    #f = open('group_data.pkl','w')
    #pickle.dump(group_data, f)
    np.save("group_data", group_data)

#gen train set
cnt = map(lambda x: len(x),group_data.values())
percentile_40 = np.percentile(cnt, 40)
train, dev ,test = train_dev_test_split(group_data, proportion = [0.6,0.2], thre = percentile_40) # should be 1000 or so
train_data, train_labels = gen_feed(train, ft_to_idx, upper_limit=12000)
del train,group_data
dev_data, dev_labels = gen_feed(dev, ft_to_idx, upper_limit=3000)
del dev
test_data, test_labels = gen_feed(test, ft_to_idx, upper_limit=3000)
del test
train_data = train_data.astype(np.float32) 
dev_data = dev_data.astype(np.float32)
test_data = test_data.astype(np.float32) 

scaler = StandardScaler()
train_data_norm = scaler.fit_transform(train_data)
dev_data_norm = scaler.transform(dev_data)
test_data = scaler.transform(test_data)




In [4]:
from itertools import product
hidden_units = [[1024,1024],[1024,512,256],[256,128,100]]#,[2048,2048],[2048,1024,512]]
batch_sizes = [128,256]#,512]
dropouts = [0.4,0.2]

#reset config, defined in previous block
config = {}
config['nclasses'] = len(np.unique(train_labels))
results = []
for hidden, batch_size, dropout in product(hidden_units,batch_sizes,dropouts):
    config['n_hidden1'] = hidden[0]
    config['n_hidden2'] = hidden[1]
    try: config['n_hidden3'] = hidden[2]
    except: config['n_hidden3'] = None
    config['dropout'] = dropout
    # Create the Estimator
    mlp_classifier = tf.estimator.Estimator(
    model_fn=mlp_model_fn, model_dir=None, params=config)

    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=5000)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=batch_size,
      num_epochs=3,
      shuffle=True)

    steps = train_data.shape[0]/batch_size
    print (steps)
    mlp_classifier.train(
      input_fn=train_input_fn,
      steps=None, # 60000
      hooks=[logging_hook])
    
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": dev_data},
        y=dev_labels,
        num_epochs=1,
        shuffle=False)

    eval_results = mlp_classifier.evaluate(input_fn=eval_input_fn)
    print ('----setting----')
    print ('hidden units:',hidden)
    print ('batch_size:', batch_size)
    print ('dropout:', dropout)
    print ('----performance----')
    print(eval_results)
    results.append((eval_results['accuracy'], hidden, batch_size, dropout))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11928d2d0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpZjEtGL', '_save_summary_steps': 100}
828.9375
Tensor("one_hot:0", shape=(?, 71), dtype=float32)
Tensor("dense_3/BiasAdd:0", shape=(?, 71), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpZjEtGL/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.00458825  0.0039679   0.04341664 ...,  0.00281583  0.00573417
   0.

In [5]:
results = sorted(results, key=lambda x: x[0], reverse=True)
print (results)

[(0.31379735, [1024, 512, 256], 128, 0.2), (0.25910133, [256, 128, 100], 128, 0.2), (0.22502203, [1024, 512, 256], 256, 0.4), (0.21631718, [1024, 1024], 256, 0.4), (0.21342732, [1024, 1024], 128, 0.2), (0.21307489, [1024, 512, 256], 256, 0.2), (0.20081057, [1024, 1024], 256, 0.2), (0.18896917, [1024, 512, 256], 128, 0.4), (0.1648282, [256, 128, 100], 128, 0.4), (0.15802643, [256, 128, 100], 256, 0.4), (0.11573568, [1024, 1024], 128, 0.4), (0.11263436, [256, 128, 100], 256, 0.2)]


In [6]:
best_param = results[0]
hidden = best_param[1]
config['n_hidden1'] = hidden[0]
config['n_hidden2'] = hidden[1]
try: config['n_hidden3'] = hidden[2]
except: config['n_hidden3'] = None
batch_size = best_param[2]
config['dropout'] = best_param[3]

# Create the Estimator
cnn_classifier = tf.estimator.Estimator(
model_fn=mlp_model_fn, model_dir=None, params=config)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=5000) 

#make full training data
train_full_data = np.vstack((train_data,dev_data))
train_full_labels = np.hstack((train_labels,dev_labels))
train_full_data = scaler.fit_transform(train_full_data)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": train_full_data},
  y=train_full_labels,
  batch_size=batch_size,
  num_epochs=3,
  shuffle=True)

steps = train_full_data.shape[0]/batch_size
cnn_classifier.train(
  input_fn=train_input_fn,
  steps=None, # 60000
  hooks=[logging_hook])

pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    num_epochs=1,
    shuffle=False)
predictions = mlp_classifier.predict(input_fn=pred_input_fn)
predictions = list(p["classes"] for p in predictions)

print ('----Best setting----')
print ('hidden units:',hidden)
print ('batch_size:', batch_size)
print ('dropout:', dropout)
print ('\n')



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x104dbcb10>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmptMauXH', '_save_summary_steps': 100}
Tensor("one_hot:0", shape=(?, 71), dtype=float32)
Tensor("dense_4/BiasAdd:0", shape=(?, 71), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmptMauXH/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.01312417  0.0067817   0.01781556 ...,  0.02245077  0.00808809
   0.01405417]

//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


IndexError: list index out of range

In [10]:
from sklearn.metrics import accuracy_score
print (accuracy_score(test_labels, predictions,normalize=True))

0.236489103264


In [34]:
target_names = np.unique(test_labels)
#target_names = np.sort(target_names)
target_names = [idx_to_ft[i] for i in target_names]
print (classification_report(test_labels, predictions))#, target_names=target_names))

             precision    recall  f1-score   support

          0       0.15      0.46      0.23      3000
          1       0.35      0.38      0.36      3000
          2       0.00      0.00      0.00      1511
          3       0.42      0.56      0.48      3000
          4       0.17      0.00      0.00      3000
          5       0.33      0.67      0.44      3000
          6       0.10      0.05      0.07      3000
          7       0.00      0.00      0.00        11
          8       0.00      0.00      0.00      1128
          9       0.01      0.01      0.01      3000
         10       0.00      0.00      0.00         3
         11       0.00      0.00      0.00        57
         12       0.00      0.00      0.00       203
         13       0.00      0.00      0.00        15
         14       0.00      0.00      0.00         7
         15       0.00      0.00      0.00        52
         16       0.00      0.00      0.00       269
         17       0.00      0.00      0.00   

In [32]:
target_names = np.unique(test_labels)
#target_names = np.sort(target_names)
[idx_to_ft[i] for i in target_names]

['text/html',
 'image/png',
 'application/rss+xml',
 'application/xhtml+xml',
 'application/pdf',
 'text/plain',
 'image/jpeg',
 'image/bmp',
 'application/xml',
 'image/gif',
 'video/x-flv',
 'image/svg+xml',
 'text/x-matlab',
 'application/vnd.ms-excel',
 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
 'application/msword',
 'image/vnd.microsoft.icon',
 'audio/mpeg',
 'application/postscript',
 'application/vnd.google-earth.kml+xml',
 'text/x-php',
 'text/calendar',
 'application/rdf+xml',
 'application/zip',
 'application/atom+xml',
 'application/x-tika-msoffice',
 'application/x-tika-ooxml',
 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'video/quicktime',
 'application/x-netcdf',
 'application/octet-stream',
 'application/x-elc',
 'application/epub+zip',
 'video/mp4',
 'application/x-msdownload; format=pe32',
 'application/x-bzip2',
 'application/x-gtar',
 'application/x-msdownload',
 'application/zlib',
 'application/vnd.google-

71